Launch interactive version: 👉👉👉 [![Try ``dyce``](https://jupyterlite.readthedocs.io/en/latest/_static/badge.svg)](https://posita.github.io/dyce-notebooks/lab?path=stack-exchange%2Faction-stress-198555%2Findex.ipynb) 👈👈👈 *[[source](https://github.com/posita/dyce-notebooks/tree/main/notebooks/stack-exchange/action-stress-198555)]*

## [``dyce``](https://posita.github.io/dyce/) solution to [“Anydice: a pool where stress dice cancel out action dice of equal or lower value and return the number and value of action dice remaining in the pool?”](https://rpg.stackexchange.com/a/198575/71245)

Once viewing this notebook in Jupyter Lab, select ``Run All Cells`` from the ``Run`` menu above.

In [1]:
# Install additional requirements if necessary
try:
    import anydyce
    import dyce
    import ipywidgets
except ImportError:
    requirements = [
        "anydyce>=0.1.4,<0.2",
        "dyce>=0.5.2,<0.6",
        "ipywidgets",
    ]
    try:
        import piplite
        await piplite.install(requirements)
    except ImportError:
        import pip
        pip.main(["install"] + requirements)

In [2]:
from dyce import P
from dyce.p import RollT

def actions_vs_stresses(action_roll: RollT, stress_roll: RollT) -> int:
    actions_not_canceled_by_stresses = []
    # We want to walk through each roll, opportunistically canceling the best action we
    # can given our maximum unspent stress. Rolls are ordered least-to-greatest, so we
    # start at the end and walk backwards, accumulating or canceling actions as we go.
    action_index = len(action_roll) - 1
    stress_index = len(stress_roll) - 1
    while action_index >= 0:
        if stress_index >= 0 and action_roll[action_index] <= stress_roll[stress_index]:
            # We have unspent stress, and our current (max unexamined) action is
            # cancelable by our current (max unspent) stress, so we decrement both
            # counters without counting the action
            action_index -= 1
            stress_index -= 1
        else:
            # Either we're out of stresses, or our current (max unexamined) action is
            # not cancelable (i.e., greater than) our current (max unspent) stress, so
            # we count that action and decrement only the action counter, leaving any
            # unspent stress for the next iteration
            actions_not_canceled_by_stresses.append(action_roll[action_index])
            action_index -= 1
    # Uncomment the following line to see the specific rolls, but this gets
    # overwhelming pretty fast. (We accumulate uncanceled actions in order of greatest-
    # to-least, above. While not strictly necessary, we reverse their order when
    # printing for consistency with roll ordering.)
    #print(f"{action_roll} vs {stress_roll} -> {actions_not_canceled_by_stresses[::-1]}")
    return len(actions_not_canceled_by_stresses)

h = P.foreach(actions_vs_stresses, action_roll=5@P(6), stress_roll=3@P(6))
print(h.format(scaled=True))

avg |    2.44
std |    0.68
var |    0.46
  2 |  65.35% |##################################################
  3 |  25.78% |###################
  4 |   7.91% |######
  5 |   0.96% |


In [ ]:
from anydyce import BreakoutType, jupyter_visualize

jupyter_visualize(
    [
        (f"{a}d6 action vs {s}d6 stress", P.foreach(actions_vs_stresses, action_roll=a @ P(6), stress_roll=s @ P(6)))
        for a in range(2, 6)
        for s in range(2, 6)
    ],
    default_breakout_type=BreakoutType.BURST,
)